In [ ]:
%load_ext dotenv
%dotenv
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pandas as pd
from common import (
    format_ax,
    query_mssql_iteratively,
    create_emission_source_query,
    PASSWORD,
)
from config import CUSTOMER, FINAL_REPORTS_PATH, FINAL_REPORT_COLUMN

In [ ]:
CUSTOMER

# Get the emission sources for the reports that have a 

In [ ]:
report_prefixes_with_error_2021 = pd.Series(
    ["763DCC", "2AE499"]
)  # From Francesco Ciocca

In [ ]:
emission_sources_with_error_2021 = query_mssql_iteratively(
    query_parameters=report_prefixes_with_error_2021,
    query_function=create_emission_source_query,
)
emission_sources_with_error_2021.info()
emission_sources_with_error_2021.head()
emission_sources_with_error_2021.nunique()

In [ ]:
emission_sources_with_error_2021.head()

In [ ]:
emission_sources_2022 = emission_sources_with_error_2021.query("Year == 2022")
len(emission_sources_2022)

In [ ]:
emission_sources_2022.to_pickle(
    f"data/non-reprocessed-emission-sources-{CUSTOMER}-errors.pickle"
)

## Get Emission Sources

In [ ]:
report_prefixes = (
    pd.read_excel(
        FINAL_REPORTS_PATH,
    )[FINAL_REPORT_COLUMN]
    .str.split("-")
    .map(lambda x: x[-1])
)

assert len(report_prefixes) == report_prefixes.nunique()

In [ ]:
report_prefixes.nunique()

In [ ]:
emission_sources = query_mssql_iteratively(
    query_parameters=report_prefixes, query_function=create_emission_source_query
)
emission_sources.info()
emission_sources.head()
emission_sources.nunique()

In [ ]:
reports_2021 = emission_sources.query("Year == 2021").ReportId.unique()
emission_sources = (
    emission_sources.set_index("ReportId").drop(reports_2021).reset_index()
)
print(len(reports_2021))

In [ ]:
assert emission_sources.Year.unique() == 2022
assert str(emission_sources.Customer.unique().squeeze()).lower() == CUSTOMER

In [ ]:
emission_sources.to_pickle(
    f"data/non-reprocessed-emission-sources-{CUSTOMER}-June.pickle"
)